<a href="https://colab.research.google.com/github/mk6749261-create/File-encryption-and-decryption/blob/main/Copy_of_notebooka958e451aa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
debasisdotcom_name_entity_recognition_ner_dataset_path = kagglehub.dataset_download('debasisdotcom/name-entity-recognition-ner-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/kaggle/input/name-entity-recognition-ner-dataset/NER dataset.csv",encoding="latin1")

In [ ]:
df.head(50)

In [ ]:
df.isnull().sum()

In [ ]:
df['Sentence #']=df['Sentence #'].fillna(method='ffill')

In [ ]:
df['Sentence #'].head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
Sentenc = df.groupby("Sentence #")["Word"].apply(list)
Sentenc = Sentenc.apply(lambda words: [w.lower() for w in words])


In [ ]:
Sentenc

In [ ]:
label = df.groupby("Sentence #")["Tag"].apply(list)


label = label.apply(lambda words: [w.upper() for w in words])

display(label.head())


In [ ]:
from sklearn.preprocessing import LabelEncoder

# جمع كل التاجات في ليست واحدة علشان نعمل fit
all_tags = [tag for doc in label for tag in doc]

# نعمل LabelEncoder
le = LabelEncoder()
le.fit(all_tags)

# نطبق التحويل على كل جملة Tags لوحدها
numeric_labels = [le.transform(doc).tolist() for doc in label]

print(label[0])          # الأصلي
print(numeric_labels[0]) # بعد التحويل


In [ ]:
import string

translator = str.maketrans("", "", string.punctuation)

def remove_punctuation(text):
    return text.translate(translator)

# تطبيق على كل جملة
Sentenc = Sentenc.apply(lambda word_list: [remove_punctuation(word) for word in word_list])

print(Sentenc.head())

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove('not')

In [ ]:
def remove_stopwords(text):
    return [word for word in text if word not in stop_words]

Sentenc = Sentenc.apply(remove_stopwords)

In [ ]:
Sentenc.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_words(text):
    return [lemmatizer.lemmatize(word) for word in text]

Sentenc = Sentenc.apply(lemmatize_words)

In [ ]:
Sentenc.head()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#from re import T
Tokenizer=Tokenizer(oov_token="<OOV>")
Tokenizer.fit_on_texts(Sentenc)

In [ ]:
word_index=Tokenizer.word_index
print(word_index)

In [ ]:
#from collections.abc import Sequence
Sequences=Tokenizer.texts_to_sequences(Sentenc)

In [ ]:
print(Sequences[0:5])

In [ ]:
padded=pad_sequences(Sequences,maxlen=50 ,padding="post",truncating='post')
print(padded[0:5])

In [ ]:
from sklearn.model_selection import train_test_split

train_sent, temp_sent, train_tags, temp_tags = train_test_split(
    Sentenc, label, test_size=0.2, random_state=42
)

val_sent, test_sent, val_tags, test_tags = train_test_split(
    temp_sent, temp_tags, test_size=0.5, random_state=42
)


In [ ]:
pip install transformers datasets seqeval


In [ ]:
from datasets import Dataset


In [ ]:
train_dict = {"tokens": train_sent, "ner_tags": train_tags}
val_dict   = {"tokens": val_sent, "ner_tags": val_tags}
test_dict  = {"tokens": test_sent, "ner_tags": test_tags}



In [ ]:
train_dataset = Dataset.from_dict(train_dict)
val_dataset   = Dataset.from_dict(val_dict)
test_dataset  = Dataset.from_dict(test_dict)


In [ ]:
from datasets import DatasetDict

datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [ ]:
print(datasets)
print(datasets["train"][0])   # أول مثال في train


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,   # مهم عشان البيانات عندنا متقسمة tokens بالفعل
        padding="max_length",       # هنحدد طول ثابت
        max_length=128              # ممكن تغيريه حسب بياناتك
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # بيرجع الـ word لكل sub-token
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:  # دي special tokens زي [CLS], [SEP], [PAD]
                label_ids.append(-100)  # -100 معناها نتجاهلها في الحساب
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # أول sub-token ياخد label الكلمة
            else:
                label_ids.append(-100)  # بقية sub-tokens نتجاهلهم
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [ ]:
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# 1) نجهز كل التاجات مرة واحدة
all_tags = [tag for doc in label for tag in doc]
le = LabelEncoder()
le.fit(all_tags)

# 2) نعيد بناء الـ Datasets لكن بالـ numeric labels
train_dataset = Dataset.from_dict({
    "tokens": train_sent,
    "ner_tags": [le.transform(tags).tolist() for tags in train_tags]
})

val_dataset = Dataset.from_dict({
    "tokens": val_sent,
    "ner_tags": [le.transform(tags).tolist() for tags in val_tags]
})

test_dataset = Dataset.from_dict({
    "tokens": test_sent,
    "ner_tags": [le.transform(tags).tolist() for tags in test_tags]
})


In [ ]:
# 1. اعملي قائمة بكل الـ labels
unique_tags = set(tag for doc in label for tag in doc)
tag2id = {tag: idx for idx, tag in enumerate(sorted(unique_tags))}
id2tag = {idx: tag for tag, idx in tag2id.items()}

# 2. حولي الداتا بتاعت الـ labels من النصوص إلى أرقام
encoded_labels = [[tag2id[tag] for tag in doc] for doc in label]

# 3. استخدمي encoded_labels بدل labels وانتِ بتمرريها للـ dataset
datasets = Dataset.from_dict({
    "tokens": Sentenc,   # التوكنز بعد التوكنايزر
    "labels": encoded_labels
})


In [ ]:
datasets = Dataset.from_dict({
    "tokens":Sentenc ,
    "ner_tags": encoded_labels
})


In [ ]:
from datasets import Dataset

# نفترض إن datasets الأساسي فيه tokens و ner_tags
datasets = Dataset.from_dict({
    "tokens": Sentenc,
    "ner_tags": encoded_labels
})

# قسميه إلى train و test و validation
datasets = datasets.train_test_split(test_size=0.2)
datasets = DatasetDict({
    "train": datasets["train"],
    "validation": datasets["test"].train_test_split(test_size=0.5)["train"],
    "test": datasets["test"].train_test_split(test_size=0.5)["test"],
})

print(datasets)


In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)


In [ ]:

print(tokenized_datasets["train"][0])

In [ ]:
print(tokenized_datasets)


In [ ]:
print(len(tokenized_datasets["train"][0]["input_ids"]))


In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",        # الموديل الأساسي
    num_labels=len(le.classes_),  # عدد التاجات بعد الـ LabelEncoder
    id2label={i: l for i, l in enumerate(le.classes_)},  # mapping رقم → label
    label2id={l: i for i, l in enumerate(le.classes_)}   # mapping label → رقم
)


In [ ]:
!pip install seqeval evaluate

import evaluate
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)

    # تجاهل -100 (tokens اللي مش معنيّة)
    true_predictions = [
        [le.classes_[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [le.classes_[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./ner_model",      # مكان حفظ الموديل
    eval_strategy="epoch",   # نقيم بعد كل Epoch
    save_strategy="epoch",         # نحفظ الموديل بعد كل Epoch
    learning_rate=2e-5,            # Learning rate مناسب لـ BERT
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,            # عدد الـ Epochs
    weight_decay=0.01,             # لتفادي overfitting
    logging_dir="./logs",          # مكان اللوجات
    logging_steps=50,
)

In [ ]:
from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="epoch",   # استخدم eval_strategy بدلاً من evaluate_during_training
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=2,
#     logging_dir="./logs",
#     logging_steps=10,
#     report_to="none"  # Add this line to disable reporting to wandb
)

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_val   = tokenized_datasets["validation"].shuffle(seed=42).select(range(200))

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=small_train,
#     eval_dataset=small_val,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
# )

# trainer.train()


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.train()